# Power iteration function

_Lavorato da: Anisa Bakiu_

## Dati di confronto

In [1]:
import numpy as np
import scipy.sparse as sparse

n=3000
A = sparse.rand(n, n, density=0.15, format="csr", random_state=42)

u0 = sparse.rand(n, 1, density=0.35, format="csr", random_state=42)
u1 = A@u0

print( (u1.T@u0)/(u0.T@u0 ) )

[[60.01905118]]


## `Met_PotenzeNorm_OK`

In [2]:
def Met_PotenzeNorm_OK(u0,A,tol,it_max):
    n_it = 0
    
    u1 = A@u0
    
    u1 = u1*(1/( np.linalg.norm( (u1).todense() ) ))
      
    lam0 = u1.T@(A@u1)/(u1.T@u1)
    u0 = u1
    
    #Collezioniamo in una lista le varie approssimazioni di lambda
    approx = []   
    approx.append(lam0)
    
    #Usiamo anche un'altra lista per memorizzare le varie stime dell'errore
    err = []
    err.append(1)
    
    while((err[-1]>tol) & (n_it <it_max) ):
        u1 = A@u0
        u1 = u1*(1/( np.linalg.norm( (u1).todense() ) ))
      
        lam = u1.T@(A@u1)/(u1.T@u1)
        
        approx.append(np.asarray(lam))
        print
        err.append(abs(lam-lam0)/(1+abs(lam)))
        lam0 = lam
        u0 = u1
        n_it = n_it+1
        
    return lam,u0,n_it,err, approx

### Tempo di esecuzione

In [3]:
import timeit

def wrapper():
    lam, u, n_it, err, approx = Met_PotenzeNorm_OK(u0, A, 1e-15, 100)

num_runs = 10
execution_time = timeit.timeit(wrapper, number=num_runs)

average_time = execution_time / num_runs
print(f'Media del tempo di esecuzione {num_runs} esecuzioni: {average_time} sec')

Media del tempo di esecuzione 10 esecuzioni: 0.1590211217000615 sec


### Errore di convergenza

In [4]:
lam,u,n_it,err, approx = Met_PotenzeNorm_OK(u0,A,1e-15,100)
print('iteration',n_it)
print(np.linalg.norm( A@u - u*lam))

iteration 12
2.3678535166359404e-13


## `Power_method`

### Spiegazione

Come primo obiettivo ho cercato di ragionare come ottimizzare: 
`u1n2_2=(u1.T@u1).todense()` perche' occupa memoria e non e' efficiente. 
Pertanto il vettore iniziale `u0` viene convertito una sola volta all'inizio in un array NumPy denso e monodimensionale tramite `toarray().ravel()`.

Lavorando con `u1 = A @ u0`, ci da' un risultato sempre sparso che richiede troppo tempo e spazio di memoria, questo si puo' risolvere di nuovo con l'utilizzo di nuovo di `toarray().ravel()`. 

L'utilizzo di `@` non e' efficiente, quindi sostituendo con `np.dot` e' una scelta migliore in vista anche delle trasformazioni che abbiamo fatto sopra. 

Nella riga `u1 = u1 * (1 / np.linalg.norm(u1.todense()))` ogni iterazione va a ripetere una conversione densa. Invece, `u0` è già un array denso, quindi `A @ u0` è un vettore denso. Pertanto, la normalizzazione viene fatta direttamente tramite operazioni del package NumPy che sono piu' veloci. 

In [5]:
import numpy as np
from numpy.linalg import norm

def power_method(u0, A, tol, it_max):
    n_it = 0
    u0 = u0.toarray().ravel()

    u1 = A @ u0
    u1 = u1 / norm(u1)

    lam0 = np.dot(u1, A @ u1) / np.dot(u1, u1)
    u0 = u1.copy()

    approx = [lam0]
    err = [1.0]

    while err[-1] > tol and n_it < it_max:
        u1 = A @ u0
        u1 /= norm(u1) # utilizzo di /= e' piu' efficiente

        Au1 = A @ u1
        lam = np.dot(u1, Au1) / np.dot(u1, u1)

        approx.append(lam)
        err.append(abs(lam - lam0) / (1 + abs(lam)))
        lam0 = lam
        u0 = u1.copy()
        n_it += 1

    return lam, u0, n_it, err, approx

### Tempo di esecuzione

In [6]:
import timeit

def wrapper():
    lam, u, n_it, err, approx = power_method(u0, A, 1e-15, 100)

num_runs = 10
execution_time = timeit.timeit(wrapper, number=num_runs)

average_time = execution_time / num_runs
print(f'Media del tempo di esecuzione {num_runs} esecuzioni: {average_time} sec')

Media del tempo di esecuzione 10 esecuzioni: 0.03730876480003644 sec


### Errore di convergenza

In [7]:
lam,u,n_it,err, approx = power_method(u0,A,1e-15,100)
print('iteration',n_it)
print(np.linalg.norm( A@u - u*lam))

iteration 10
2.3294701090078497e-12


# Conclusione

Alla fine possiamo concludere che l'efficienza della funzione e' aumentata grazie alla conversione iniziale del vettore sparso in un array NumPy denso monodimensionale (il concetto di flattening) tramite `.toarray().ravel().`

Inoltre, riducendo la tolleranza `(tol)`, e' possibile ottenere velocemente un'approssimazione dell'autovalore dominante molto vicina a quella restituita di `scipy`.

# Materiale

1. https://en.wikipedia.org/wiki/Power_iteration
2. https://ergodic.ugr.es/cphys/LECCIONES/FORTRAN/power_method.pdf
3. https://realpython.com/python-flatten-list/